In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, balanced_accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import mutual_info_regression
from sklearn.utils import resample
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import os

# === 1. Імпорт даних та обробка пропусків ===
file_train_path = '/kaggle/input/ml-fundamentals-and-applications-2025-01-09/final_proj_data.csv'
file_test_path = '/kaggle/input/ml-fundamentals-and-applications-2025-01-09/final_proj_test.csv'

train_data = pd.read_csv(file_train_path)
valid_data = pd.read_csv(file_test_path)

assert valid_data.shape[0] == 2500, f"Валідаційні дані мають {valid_data.shape[0]} рядків, а не 2500!"

print("Дані для навчання")
print(train_data.head())
print(f"Кількість рядків у тренувальних даних: {train_data.shape[0]}")

print("\nДані для валідації")
print(valid_data.head())
print(f"Кількість рядків у валідаційних даних: {valid_data.shape[0]}")

# Визначення числових і текстових колонок
numeric_cols = train_data.select_dtypes(include=['number']).columns.tolist()
categorical_cols = train_data.select_dtypes(include=['object']).columns.tolist()

# Видалення 'y' з числових колонок, якщо вона є
if 'y' in numeric_cols:
    numeric_cols.remove('y')

# Заповнення числових колонок
train_data[numeric_cols] = train_data[numeric_cols].fillna(train_data[numeric_cols].mean())
valid_data[numeric_cols] = valid_data[numeric_cols].fillna(valid_data[numeric_cols].mean())

# Заповнення текстових колонок
train_data[categorical_cols] = train_data[categorical_cols].fillna('missing')
valid_data[categorical_cols] = valid_data[categorical_cols].fillna('missing')

# Перевірка після обробки
print("\nПропуски в train_data після обробки:")
print(train_data.isnull().sum().sum())

print("\nПропуски в valid_data після обробки:")
print(valid_data.isnull().sum().sum())


In [ ]:
# === 2. Балансування класів ===
class_counts = train_data['y'].value_counts()
print("Кількість кожного класу в тренувальних даних:")
print(class_counts)

data_class_0 = train_data[train_data['y'] == 0]
data_class_1 = train_data[train_data['y'] == 1]

data_class_0_undersampled = resample(data_class_0, replace=False, n_samples=len(data_class_1), random_state=42)

train_data = pd.concat([data_class_0_undersampled, data_class_1])
print("Новий баланс класів:")
print(train_data['y'].value_counts())

In [ ]:
# === 3. Видалення пропусків ===
def clean_data(data):
    column_threshold = 0.9  # Видаляємо колонки, де більше 90% пропусків
    row_threshold = 0.7  # Видаляємо рядки, де більше 70% пропусків

    missing_values = data.isnull().sum() / len(data)
    columns_to_drop = missing_values[missing_values > column_threshold].index
    data = data.drop(columns=columns_to_drop, errors='ignore')

    row_missing = data.isnull().mean(axis=1)
    rows_to_drop = row_missing[row_missing > row_threshold].index
    data = data.drop(index=rows_to_drop)
    return data

train_data = clean_data(train_data)
valid_data = clean_data(valid_data)
print(f"Кількість рядків у valid_data після clean_data: {valid_data.shape[0]}")

In [ ]:
# === 4. Видалення дублікатів ===
def drop_duplicates(data):
    return data.drop_duplicates()

train_data = drop_duplicates(train_data)
valid_data = drop_duplicates(valid_data)

In [ ]:
# === 5. Визначення дискретних ознак ===
discrete_features = [col for col in train_data.columns if train_data[col].dtype == 'object']
if 'y' in discrete_features:
    discrete_features.remove('y')
print("Оновлений перелік дискретних ознак:", discrete_features)

for col in discrete_features:
    train_data[col] = train_data[col].astype('category')
    if 'missing' not in train_data[col].cat.categories:
        train_data[col] = train_data[col].cat.add_categories(['missing'])
    train_data[col] = train_data[col].fillna('missing')

    valid_data[col] = valid_data[col].astype('category')
    if 'missing' not in valid_data[col].cat.categories:
        valid_data[col] = valid_data[col].cat.add_categories(['missing'])
    valid_data[col] = valid_data[col].fillna('missing')
    valid_data[col] = valid_data[col].cat.set_categories(train_data[col].cat.categories)

In [ ]:
# === 6. Вибір ознак за показником взаємної інформації ===
autos_encoded = train_data.copy()
for col in discrete_features:
    autos_encoded[col], _ = autos_encoded[col].factorize()

X = autos_encoded.drop(columns=['y'])
y = autos_encoded['y']

print("Перевірка на NaN перед обробкою:")
print(X.isna().sum())
X = X.fillna(X.mean())

mi_scores = mutual_info_regression(X, y, discrete_features=[X.columns.get_loc(col) for col in discrete_features])
mi_scores = pd.Series(mi_scores, name='MI Scores', index=X.columns).sort_values(ascending=False)
print("Показники взаємної інформації:")
print(mi_scores)

selected_features = mi_scores.head(50).index.tolist()
train_data = train_data[selected_features + ['y']]
valid_data = valid_data[selected_features]

In [ ]:
# === 7. Кодування категоріальних ознак ===

# Перевірка та фільтрація дискретних ознак
discrete_features = [col for col in discrete_features if col in train_data.columns]

# Узгодження категорій між тренувальними та валідаційними даними
for col in discrete_features:
    if col in valid_data.columns:
        valid_data[col] = pd.Categorical(valid_data[col], categories=train_data[col].cat.categories)
    else:
        print(f"Попередження: колонка '{col}' відсутня у valid_data. Пропускаємо.")

# Кодування категоріальних змінних
train_data_encoded = pd.get_dummies(train_data, columns=discrete_features)
valid_data_encoded = pd.get_dummies(valid_data, columns=discrete_features)

# Вирівнювання колонок між тренувальними та валідаційними даними
valid_data_encoded = valid_data_encoded.reindex(columns=train_data_encoded.columns.drop('y'), fill_value=0)

# Перевірка кількості рядків у valid_data_encoded після обробки
print(f"Кількість рядків у valid_data_encoded до виправлення: {valid_data_encoded.shape[0]}")

# Додавання відсутніх рядків, якщо їх кількість менша за 2500
if valid_data_encoded.shape[0] < 2500:
    missing_rows = 2500 - valid_data_encoded.shape[0]
    additional_rows = pd.DataFrame(0, index=range(missing_rows), columns=valid_data_encoded.columns)
    valid_data_encoded = pd.concat([valid_data_encoded, additional_rows], ignore_index=True)

# Перевірка після виправлення
print(f"Кількість рядків у valid_data_encoded після виправлення: {valid_data_encoded.shape[0]}")

In [ ]:
# === 8. Балансування даних за допомогою SMOTE ===
X = train_data_encoded.drop(columns=['y'])
y = train_data_encoded['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = X_train.astype(float)
X_test = X_test.astype(float)

X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

sm = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = sm.fit_resample(X_train, y_train)

print("Розміри після застосування SMOTE:")
print(f"X_train_resampled: {X_train_resampled.shape}")
print(f"y_train_resampled: {y_train_resampled.shape}")

In [ ]:
# === 9. Масштабування даних ===
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_resampled)
X_test_scaled = scaler.transform(X_test)
X_valid_scaled = scaler.transform(valid_data_encoded)

In [ ]:
# === 10. Моделювання та вибір найкращої моделі ===
best_model = None
best_model_name = None
best_balanced_acc = 0.0

models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    'LightGBM': LGBMClassifier(random_state=42),
    'CatBoost': CatBoostClassifier(verbose=0, random_state=42)
}

for model_name, model in models.items():
    print(f"\nНавчання моделі: {model_name}")
    model.fit(X_train_scaled, y_train_resampled)
    y_pred = model.predict(X_test_scaled)
    balanced_acc = balanced_accuracy_score(y_test, y_pred)
    print(f"Balanced Accuracy: {balanced_acc:.4f}")
    print(classification_report(y_test, y_pred))
    
    # Зберігаємо найкращу модель
    if balanced_acc > best_balanced_acc:
        best_model = model
        best_model_name = model_name
        best_balanced_acc = balanced_acc

print(f"\nНайкраща модель: {best_model_name} з Balanced Accuracy: {best_balanced_acc:.4f}")



In [ ]:
# === 11. Передбачення для валідаційних даних ===

# Перевірка та заповнення пропущених значень у valid_data_encoded
if valid_data_encoded.isna().sum().sum() > 0:
    print("Пропущені значення в valid_data_encoded виявлено! Заповнюємо нулями.")
    valid_data_encoded = valid_data_encoded.fillna(0)

# Масштабування даних
X_valid_scaled = scaler.transform(valid_data_encoded)

# Перевірка наявності NaN після масштабування
print("Кількість NaN у X_valid_scaled:", np.isnan(X_valid_scaled).sum())
assert not np.isnan(X_valid_scaled).any(), "У X_valid_scaled залишаються NaN!"

# Використання найкращої моделі для передбачення
y_valid_pred = best_model.predict(X_valid_scaled)

# Збереження результатів
result_df = pd.DataFrame({'index': np.arange(len(y_valid_pred)), 'y': y_valid_pred})
result_df.to_csv('submission.csv', index=False)
print(f"Прогнози збережено у файл submission.csv за допомогою моделі {best_model_name}")